In [11]:
#libraries
import keras
import tensorflow as tf
from keras import backend as K
import cv2
import os
import numpy as np
from keras.optimizers import Adam,SGD
from keras.models import model_from_json, load_model
from keras.layers import Input, Dense
from keras.models import Model,Sequential
from sklearn.model_selection import train_test_split
from keras.layers import Convolution2D as Conv2D
from keras.layers.convolutional import Deconv2D as Conv2DTranspose
from keras.layers import Lambda, Input, Dense, MaxPooling2D, BatchNormalization,Input
from keras.layers import UpSampling2D, Dropout, Flatten, Reshape, RepeatVector, LeakyReLU,Activation
from keras.callbacks import ModelCheckpoint
from keras.losses import mse, binary_crossentropy
from keras.callbacks import EarlyStopping
seed = 7
np.random.seed(seed)
from keras.callbacks import CSVLogger
from keras.callbacks import History 
hist = History()

In [12]:
os.environ["CUDA_VISIBLE_DEVICES"]="1,2"#Setting the script to run on GPU:1,2

In [13]:
import os
import sys
import cv2
import csv
import glob
import numpy as np
import time
from sklearn.utils import shuffle

# path to USB
USBPath = "/home/scope/Shreyas/conferences/NoveltyDetection/Code/Carla/Motivation/"
trainingFolders = ["sunny-train"]
history = CSVLogger(USBPath + '/kerasloss.csv', append=True, separator=';')

#Load complete input images without shuffling
def load_images(paths):
    numImages = 0
    inputs = []
    for path in paths:
        numFiles = len(glob.glob1(path,'*.png'))
        numImages += numFiles
        for img in glob.glob(path+'*.png'):
            img = cv2.imread(img)
            img = cv2.resize(img, (200,66))
            img = img /255.
            inputs.append(img)
    #inpu = shuffle(inputs)
    print("Total number of images:%d" %(numImages))
    return inputs

def createFolderPaths(folders):
    paths = []
    for folder in folders:
        path = USBPath + folder + '/'
        paths.append(path)
    return paths

def load_training_images():
    paths = createFolderPaths(trainingFolders)
    return load_images(paths)

def load_steering_value():
    miny= -1
    maxy= 1
    Y=[]
    dataset = USBPath + '/sunny-train.csv'
    with open(dataset, 'rt') as csvfile:
          reader = csv.reader(csvfile)
          for row in reader:
            output=[]
            x=(float(row[0])-miny)/(maxy-miny)
            output.append(x)
            Y.append(output)
    return Y        

In [14]:
#Loading images from the datasets
image_input = load_training_images()
steer_output = load_steering_value()
print("Total Steering values:%d"%len(steer_output))
image_input = np.array(image_input)
steer_output = np.array(steer_output)

Total number of images:4999
Total Steering values:4999


In [29]:
#CNN-VAE model.
def createModel():
    model = Sequential([
        Conv2D(24, kernel_size=(5,5), strides=(2,2), activation='relu', input_shape=(66,200,3)),
        BatchNormalization(),
        Conv2D(36, kernel_size=(5,5), strides=(2,2), activation='relu'),
        BatchNormalization(),
        Conv2D(48, kernel_size=(5,5), strides=(2,2), activation='relu'),
        BatchNormalization(),
        Conv2D(64, kernel_size=(3,3), strides=(1,1), activation='relu'),
        BatchNormalization(),
        Conv2D(64, kernel_size=(3,3), strides=(1,1), activation='relu'),
        BatchNormalization(),
        Flatten(),
        Dense(100, activation='relu'),
        BatchNormalization(),
        Dense(50, activation='relu'),
        BatchNormalization(),
        Dense(10, activation='relu'),
        BatchNormalization(),
        Dense(1)
    ])
    return model

#training the model with different hyperparameters
def trainModel(model, X, Y,e,b):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.05,random_state=42)
    #sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    #model.compile(optimizer=sgd, loss="mse", metrics=['mae'])
    adam = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(loss='mse', optimizer=adam)
    callbacks_list = [
    EarlyStopping(monitor='loss', patience=10, verbose=0),
    ]
    hist=model.fit(X_train, Y_train, epochs=e, batch_size=b,validation_data=(X_test, Y_test),callbacks=callbacks_list, verbose=2)
    return hist

#function for arranging list in descending order
def takeFirst(elem):
    return elem[2]


In [30]:
selection=[]
epochs = [50,100,150,200,250]
batches = [16,32,64,128,256,512]
model = createModel()# create model
for epoch_val in epochs:
    loss = []
    for batch_val in batches:
        print("**********************epoch=%d,batch=%d*********************"%(epoch_val,batch_val))
        loss_val=trainModel(model, image_input, steer_output, epoch_val, batch_val)#train model with new hyperparameters
        loss.append((epoch_val,batch_val, loss_val.history['loss'][-1]))#extract and append loss from last epoch
        loss.sort(key=takeFirst,reverse=False)#sort list to get the hyperparameters resulting in least loss
        print(loss[0])
    selection.append(loss[0])
#print(selection)
selection.sort(key=takeFirst,reverse=False)
print(selection)
    
        

**********************epoch=50,batch=16*********************
Train on 4749 samples, validate on 250 samples
Epoch 1/50
 - 10s - loss: 0.6163 - val_loss: 0.2147
Epoch 2/50
 - 4s - loss: 0.2734 - val_loss: 0.1380
Epoch 3/50
 - 4s - loss: 0.1573 - val_loss: 0.1117
Epoch 4/50
 - 4s - loss: 0.0962 - val_loss: 0.0507
Epoch 5/50
 - 4s - loss: 0.0666 - val_loss: 0.0366
Epoch 6/50
 - 4s - loss: 0.0488 - val_loss: 0.0235
Epoch 7/50
 - 4s - loss: 0.0370 - val_loss: 0.0175
Epoch 8/50
 - 4s - loss: 0.0322 - val_loss: 0.0200
Epoch 9/50
 - 4s - loss: 0.0262 - val_loss: 0.0145
Epoch 10/50
 - 4s - loss: 0.0221 - val_loss: 0.0122
Epoch 11/50
 - 4s - loss: 0.0215 - val_loss: 0.0134
Epoch 12/50
 - 4s - loss: 0.0194 - val_loss: 0.0124
Epoch 13/50
 - 4s - loss: 0.0161 - val_loss: 0.0078
Epoch 14/50
 - 4s - loss: 0.0143 - val_loss: 0.0084
Epoch 15/50
 - 4s - loss: 0.0136 - val_loss: 0.0060
Epoch 16/50
 - 4s - loss: 0.0103 - val_loss: 0.0056
Epoch 17/50
 - 4s - loss: 0.0098 - val_loss: 0.0056
Epoch 18/50
 - 4

Epoch 2/50
 - 1s - loss: 4.5092e-04 - val_loss: 0.0025
Epoch 3/50
 - 1s - loss: 4.7895e-04 - val_loss: 0.0027
Epoch 4/50
 - 1s - loss: 4.3262e-04 - val_loss: 0.0025
Epoch 5/50
 - 1s - loss: 4.1380e-04 - val_loss: 0.0025
Epoch 6/50
 - 1s - loss: 4.5968e-04 - val_loss: 0.0024
Epoch 7/50
 - 1s - loss: 4.1333e-04 - val_loss: 0.0026
Epoch 8/50
 - 1s - loss: 4.0383e-04 - val_loss: 0.0024
Epoch 9/50
 - 1s - loss: 3.8914e-04 - val_loss: 0.0025
Epoch 10/50
 - 1s - loss: 3.9828e-04 - val_loss: 0.0025
Epoch 11/50
 - 1s - loss: 4.2174e-04 - val_loss: 0.0025
Epoch 12/50
 - 1s - loss: 3.9263e-04 - val_loss: 0.0024
Epoch 13/50
 - 1s - loss: 3.8421e-04 - val_loss: 0.0026
Epoch 14/50
 - 1s - loss: 3.7456e-04 - val_loss: 0.0024
Epoch 15/50
 - 1s - loss: 3.7435e-04 - val_loss: 0.0025
Epoch 16/50
 - 1s - loss: 3.8992e-04 - val_loss: 0.0025
Epoch 17/50
 - 1s - loss: 3.8395e-04 - val_loss: 0.0025
Epoch 18/50
 - 1s - loss: 3.6879e-04 - val_loss: 0.0024
Epoch 19/50
 - 1s - loss: 3.8459e-04 - val_loss: 0.0026


 - 3s - loss: 5.2751e-04 - val_loss: 0.0025
Epoch 10/100
 - 3s - loss: 5.4227e-04 - val_loss: 0.0026
Epoch 11/100
 - 3s - loss: 5.2700e-04 - val_loss: 0.0025
Epoch 12/100
 - 3s - loss: 4.9377e-04 - val_loss: 0.0024
Epoch 13/100
 - 3s - loss: 5.1317e-04 - val_loss: 0.0024
Epoch 14/100
 - 3s - loss: 5.8714e-04 - val_loss: 0.0024
Epoch 15/100
 - 3s - loss: 5.5385e-04 - val_loss: 0.0024
Epoch 16/100
 - 3s - loss: 4.9837e-04 - val_loss: 0.0024
Epoch 17/100
 - 3s - loss: 4.8660e-04 - val_loss: 0.0025
Epoch 18/100
 - 3s - loss: 4.9066e-04 - val_loss: 0.0022
Epoch 19/100
 - 3s - loss: 4.8041e-04 - val_loss: 0.0022
Epoch 20/100
 - 3s - loss: 4.7744e-04 - val_loss: 0.0023
Epoch 21/100
 - 3s - loss: 4.8631e-04 - val_loss: 0.0023
Epoch 22/100
 - 3s - loss: 4.7281e-04 - val_loss: 0.0024
Epoch 23/100
 - 3s - loss: 5.0515e-04 - val_loss: 0.0024
Epoch 24/100
 - 3s - loss: 4.8817e-04 - val_loss: 0.0024
Epoch 25/100
 - 3s - loss: 4.6975e-04 - val_loss: 0.0023
Epoch 26/100
 - 3s - loss: 4.6958e-04 - val_

Epoch 64/100
 - 1s - loss: 1.6964e-04 - val_loss: 0.0025
Epoch 65/100
 - 1s - loss: 1.7533e-04 - val_loss: 0.0028
Epoch 66/100
 - 1s - loss: 1.7339e-04 - val_loss: 0.0025
Epoch 67/100
 - 1s - loss: 1.8019e-04 - val_loss: 0.0025
Epoch 68/100
 - 1s - loss: 1.7261e-04 - val_loss: 0.0027
Epoch 69/100
 - 1s - loss: 1.8163e-04 - val_loss: 0.0025
Epoch 70/100
 - 1s - loss: 1.7032e-04 - val_loss: 0.0025
Epoch 71/100
 - 1s - loss: 1.7522e-04 - val_loss: 0.0025
Epoch 72/100
 - 1s - loss: 1.8257e-04 - val_loss: 0.0026
Epoch 73/100
 - 1s - loss: 1.7931e-04 - val_loss: 0.0025
Epoch 74/100
 - 1s - loss: 1.8232e-04 - val_loss: 0.0026
(100, 256, 0.0001823203620607448)
**********************epoch=100,batch=512*********************
Train on 4749 samples, validate on 250 samples
Epoch 1/100
 - 9s - loss: 1.5677e-04 - val_loss: 0.0025
Epoch 2/100
 - 1s - loss: 1.4783e-04 - val_loss: 0.0025
Epoch 3/100
 - 1s - loss: 1.5112e-04 - val_loss: 0.0025
Epoch 4/100
 - 1s - loss: 1.3545e-04 - val_loss: 0.0025
Epoch

 - 4s - loss: 5.9502e-04 - val_loss: 0.0024
Epoch 43/150
 - 5s - loss: 5.7845e-04 - val_loss: 0.0029
Epoch 44/150
 - 5s - loss: 5.5084e-04 - val_loss: 0.0024
Epoch 45/150
 - 5s - loss: 5.5296e-04 - val_loss: 0.0027
Epoch 46/150
 - 5s - loss: 5.7118e-04 - val_loss: 0.0026
Epoch 47/150
 - 4s - loss: 5.6638e-04 - val_loss: 0.0025
Epoch 48/150
 - 4s - loss: 5.7664e-04 - val_loss: 0.0033
Epoch 49/150
 - 5s - loss: 5.5187e-04 - val_loss: 0.0029
Epoch 50/150
 - 4s - loss: 5.4092e-04 - val_loss: 0.0023
Epoch 51/150
 - 4s - loss: 5.6010e-04 - val_loss: 0.0024
Epoch 52/150
 - 5s - loss: 6.7398e-04 - val_loss: 0.0024
Epoch 53/150
 - 5s - loss: 6.2861e-04 - val_loss: 0.0022
Epoch 54/150
 - 4s - loss: 5.9653e-04 - val_loss: 0.0024
Epoch 55/150
 - 4s - loss: 5.6862e-04 - val_loss: 0.0026
Epoch 56/150
 - 4s - loss: 5.6563e-04 - val_loss: 0.0024
Epoch 57/150
 - 4s - loss: 6.1559e-04 - val_loss: 0.0030
Epoch 58/150
 - 4s - loss: 6.1552e-04 - val_loss: 0.0029
Epoch 59/150
 - 4s - loss: 5.9713e-04 - val_

Epoch 17/150
 - 1s - loss: 1.6115e-04 - val_loss: 0.0025
Epoch 18/150
 - 1s - loss: 1.7100e-04 - val_loss: 0.0024
Epoch 19/150
 - 1s - loss: 1.5664e-04 - val_loss: 0.0026
Epoch 20/150
 - 1s - loss: 1.4048e-04 - val_loss: 0.0024
Epoch 21/150
 - 1s - loss: 1.5986e-04 - val_loss: 0.0025
Epoch 22/150
 - 1s - loss: 1.4462e-04 - val_loss: 0.0024
Epoch 23/150
 - 1s - loss: 1.2699e-04 - val_loss: 0.0024
Epoch 24/150
 - 1s - loss: 1.3635e-04 - val_loss: 0.0024
Epoch 25/150
 - 1s - loss: 1.3231e-04 - val_loss: 0.0024
Epoch 26/150
 - 1s - loss: 1.2008e-04 - val_loss: 0.0027
Epoch 27/150
 - 1s - loss: 1.4226e-04 - val_loss: 0.0026
Epoch 28/150
 - 1s - loss: 1.3542e-04 - val_loss: 0.0023
Epoch 29/150
 - 1s - loss: 1.3662e-04 - val_loss: 0.0025
Epoch 30/150
 - 1s - loss: 1.4821e-04 - val_loss: 0.0027
Epoch 31/150
 - 1s - loss: 1.8292e-04 - val_loss: 0.0024
Epoch 32/150
 - 1s - loss: 1.5019e-04 - val_loss: 0.0028
Epoch 33/150
 - 1s - loss: 1.5508e-04 - val_loss: 0.0024
Epoch 34/150
 - 1s - loss: 1.41

 - 3s - loss: 3.0061e-04 - val_loss: 0.0024
Epoch 32/200
 - 3s - loss: 2.5644e-04 - val_loss: 0.0023
Epoch 33/200
 - 3s - loss: 2.6761e-04 - val_loss: 0.0021
Epoch 34/200
 - 3s - loss: 2.9021e-04 - val_loss: 0.0023
Epoch 35/200
 - 3s - loss: 2.7573e-04 - val_loss: 0.0031
Epoch 36/200
 - 3s - loss: 2.6731e-04 - val_loss: 0.0023
Epoch 37/200
 - 3s - loss: 2.7372e-04 - val_loss: 0.0026
(200, 32, 0.0002737198675384325)
**********************epoch=200,batch=64*********************
Train on 4749 samples, validate on 250 samples
Epoch 1/200
 - 11s - loss: 2.1456e-04 - val_loss: 0.0023
Epoch 2/200
 - 2s - loss: 2.0095e-04 - val_loss: 0.0025
Epoch 3/200
 - 2s - loss: 1.8132e-04 - val_loss: 0.0024
Epoch 4/200
 - 2s - loss: 1.8745e-04 - val_loss: 0.0022
Epoch 5/200
 - 2s - loss: 2.0001e-04 - val_loss: 0.0023
Epoch 6/200
 - 2s - loss: 1.8899e-04 - val_loss: 0.0023
Epoch 7/200
 - 2s - loss: 1.9189e-04 - val_loss: 0.0025
Epoch 8/200
 - 2s - loss: 1.7385e-04 - val_loss: 0.0022
Epoch 9/200
 - 2s - los

Epoch 37/200
 - 1s - loss: 3.1578e-05 - val_loss: 0.0027
Epoch 38/200
 - 2s - loss: 3.5223e-05 - val_loss: 0.0034
Epoch 39/200
 - 1s - loss: 3.4055e-05 - val_loss: 0.0025
Epoch 40/200
 - 2s - loss: 2.9445e-05 - val_loss: 0.0027
Epoch 41/200
 - 1s - loss: 2.8451e-05 - val_loss: 0.0028
Epoch 42/200
 - 2s - loss: 3.3980e-05 - val_loss: 0.0024
Epoch 43/200
 - 2s - loss: 3.2426e-05 - val_loss: 0.0024
Epoch 44/200
 - 2s - loss: 3.0056e-05 - val_loss: 0.0024
Epoch 45/200
 - 2s - loss: 2.9962e-05 - val_loss: 0.0025
Epoch 46/200
 - 2s - loss: 2.9497e-05 - val_loss: 0.0024
Epoch 47/200
 - 2s - loss: 2.3249e-05 - val_loss: 0.0024
Epoch 48/200
 - 2s - loss: 3.0584e-05 - val_loss: 0.0025
Epoch 49/200
 - 1s - loss: 2.8066e-05 - val_loss: 0.0026
Epoch 50/200
 - 2s - loss: 3.1861e-05 - val_loss: 0.0025
Epoch 51/200
 - 1s - loss: 3.0147e-05 - val_loss: 0.0025
Epoch 52/200
 - 1s - loss: 3.1043e-05 - val_loss: 0.0024
Epoch 53/200
 - 2s - loss: 3.4871e-05 - val_loss: 0.0025
Epoch 54/200
 - 2s - loss: 3.19

Epoch 42/250
 - 1s - loss: 9.1500e-05 - val_loss: 0.0023
(250, 128, 9.14995526511362e-05)
**********************epoch=250,batch=256*********************
Train on 4749 samples, validate on 250 samples
Epoch 1/250
 - 13s - loss: 6.4139e-05 - val_loss: 0.0024
Epoch 2/250
 - 1s - loss: 5.5267e-05 - val_loss: 0.0023
Epoch 3/250
 - 2s - loss: 5.3641e-05 - val_loss: 0.0023
Epoch 4/250
 - 1s - loss: 4.5424e-05 - val_loss: 0.0023
Epoch 5/250
 - 2s - loss: 4.5512e-05 - val_loss: 0.0023
Epoch 6/250
 - 1s - loss: 4.0257e-05 - val_loss: 0.0029
Epoch 7/250
 - 2s - loss: 4.0645e-05 - val_loss: 0.0024
Epoch 8/250
 - 1s - loss: 3.7580e-05 - val_loss: 0.0024
Epoch 9/250
 - 2s - loss: 4.5211e-05 - val_loss: 0.0024
Epoch 10/250
 - 1s - loss: 4.3213e-05 - val_loss: 0.0023
Epoch 11/250
 - 2s - loss: 4.3347e-05 - val_loss: 0.0023
Epoch 12/250
 - 1s - loss: 4.2574e-05 - val_loss: 0.0023
Epoch 13/250
 - 2s - loss: 4.1622e-05 - val_loss: 0.0024
Epoch 14/250
 - 1s - loss: 3.9767e-05 - val_loss: 0.0023
Epoch 15/2